In [1]:
import json
import pandas as pd
import csv
import os


In [5]:
import pandas as pd
# Charger le fichier CSV
df1 = pd.read_csv('Q2_2data.csv', dtype={10: str}, low_memory=False)
print(df1['eventType'].unique())



['shot-on-goal' 'goal']


In [6]:
# Afficher uniquement les lignes où eventType == 'goal' avec les colonnes eventType, emptyNet et strength
goals_with_empty_net = df1[df1['isGoal'] == 1][['eventType', 'emptyNet', 'strength']]
print(goals_with_empty_net)



       eventType  emptyNet strength
6           goal         0       EV
11          goal         0       EV
14          goal         0       EV
15          goal         0       EV
24          goal         0       EV
...          ...       ...      ...
384902      goal         0       EV
384910      goal         0       EV
384915      goal         0       EV
384923      goal         0       EV
384953      goal         0       EV

[36558 rows x 3 columns]


In [7]:
# Vérifier les valeurs uniques de 'shotType' pour confirmer le filtrage des goals
print('event Types: ', df1['eventType'].unique())  # 0: Non-Goal, 1: Goal
print(df1.columns)

event Types:  ['shot-on-goal' 'goal']
Index(['gameId', 'season', 'teamHomeId', 'teamHome', 'teamAwayId', 'teamAway',
       'event_id', 'eventType', 'eventTeamId', 'eventTeam', 'zoneCode',
       'period', 'periodTime', 'coordinateX', 'coordinateY', 'emptyNet',
       'strength', 'distance', 'distance_round', 'angle', 'isGoal'],
      dtype='object')


In [8]:
print(df1.tail())
print('-------------------------------------')


            gameId    season  teamHomeId   teamHome  teamAwayId   teamAway  \
384971  2020030415  20202021          14  Lightning           8  Canadiens   
384972  2020030415  20202021          14  Lightning           8  Canadiens   
384973  2020030415  20202021          14  Lightning           8  Canadiens   
384974  2020030415  20202021          14  Lightning           8  Canadiens   
384975  2020030415  20202021          14  Lightning           8  Canadiens   

        event_id     eventType  eventTeamId  eventTeam  ... period  \
384971       692  shot-on-goal           14  Lightning  ...      3   
384972       693  shot-on-goal           14  Lightning  ...      3   
384973       697  shot-on-goal            8  Canadiens  ...      3   
384974       698  shot-on-goal            8  Canadiens  ...      3   
384975       699  shot-on-goal            8  Canadiens  ...      3   

        periodTime coordinateX  coordinateY  emptyNet  strength   distance  \
384971       16:19        57.0  

In [9]:
print(df1.columns)

Index(['gameId', 'season', 'teamHomeId', 'teamHome', 'teamAwayId', 'teamAway',
       'event_id', 'eventType', 'eventTeamId', 'eventTeam', 'zoneCode',
       'period', 'periodTime', 'coordinateX', 'coordinateY', 'emptyNet',
       'strength', 'distance', 'distance_round', 'angle', 'isGoal'],
      dtype='object')


In [ ]:
import numpy as np
import math

def get_distance(x1, y1, x2, y2):
    try:
        distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    except:
        distance = 0

    return distance

In [11]:
#Get time differece
import datetime as dt
def get_time_diff(t1, t2):
    start_dt = dt.datetime.strptime(t1, '%M:%S')
    end_dt = dt.datetime.strptime(t2, '%M:%S')
    diff = (end_dt - start_dt) 
    return diff.seconds 

In [ ]:
import os
import json
import pandas as pd


class NHLPlayByPlayExtractor:
    
    def __init__(self, json_file):
        with open(json_file, 'r', encoding='utf-8') as file:
            self.data = json.load(file)

    

        
    def extract_events(self):
        events_list = []
        gameId = self.data.get('id')
        season = self.data.get('season')
        plays = self.data.get('plays', [])
        teamHomeId = self.data['homeTeam']['id']
        teamAwayId = self.data['awayTeam']['id']
        homeDefendingSidePeriod = {1: None, 2: None, 3: None}
      
        
        for i in range(len(plays)):
            play = plays[i]
            if play["typeDescKey"] in ['shot-on-goal', 'goal']:
                last_details = plays[i-1].get('details', {}) if i > 0 else {}
                details = play.get('details', {})
                eventTeamId = details.get('eventOwnerTeamId', None)
                zoneCode = details.get('zoneCode', None)

                lastEventTeamId = last_details.get('eventOwnerTeamId', None)
                period = play['periodDescriptor']['number']
                lastPeriod = plays[i-1]['periodDescriptor']['number'] if i > 0 else None
                coordinateX = details.get('xCoord', None)
                coordinateY = details.get('yCoord', None)
                shotType = play['details'].get('shotType', None)
                eventType = play["typeDescKey"]
                periodTime = play['timeInPeriod']
              
                
                
                # Extraction du situationCode et des informations pertinentes
                situationCode = play.get('situationCode', None)

                if situationCode:
                  away_goalie = situationCode[0]  # 1 ou 0 pour le gardien extérieur
                  away_skaters = int(situationCode[1])  # Nombre de joueurs extérieurs
                  home_skaters = int(situationCode[2])  # Nombre de joueurs à domicile
                  home_goalie = situationCode[3]  # 1 ou 0 pour le gardien à domicile

                # Calcul de empty_net
                  if eventTeamId== self.data['awayTeam']['id']:
                    # L'équipe qui effectue l'événement est l'équipe extérieure
                    if home_goalie == '1':
                       emptyNet = 0  # Le gardien de l'équipe à domicile est dans son filet
                    else:
                       emptyNet = 1  # Le gardien de l'équipe à domicile a quitté son filet
                  elif eventTeamId == self.data['homeTeam']['id']:
                      # L'équipe qui effectue l'événement est l'équipe à domicile
                    if away_goalie == '1':
                       emptyNet = 0  # Le gardien de l'équipe extérieure est dans son filet
                    else:
                       emptyNet = 1  # Le gardien de l'équipe extérieure a quitté son filet
                  else:
                      emptyNet = None  # Cas non traité

              
                else:
                  emptyNet = None
                  
                
                lastCoordinateX = last_details.get('xCoord', None)
                lastCoordinateY = last_details.get('yCoord', None)
                if lastCoordinateX is None :
                    lastCoordinateX = 0
                if lastCoordinateY is None :
                    lastCoordinateY = 0
                lastEvent = plays[i-1].get("typeDescKey", None)
                periodTime= plays[i].get('timeInPeriod', None)
                lastPeriodTime = plays[i-1].get('timeInPeriod', None)

                lastDistance = get_distance(coordinateX, coordinateY, lastCoordinateX, lastCoordinateY)
                timeLastEvent = get_time_diff(periodTime, lastPeriodTime)
                rebound=False
                rebound = lastEvent == "shot-on-goal"
                speed = lastDistance / timeLastEvent if timeLastEvent else 0
                

                
                events_list.append([
                        gameId, season, period, periodTime, eventType, shotType, coordinateX, coordinateY, emptyNet, eventTeamId, teamHomeId, teamAwayId, away_skaters, home_skaters, zoneCode,
                        lastEvent, lastCoordinateX, lastCoordinateY, lastDistance,timeLastEvent, speed, rebound
                    ])

        columns = [
            'gameId', 'season', 'period', 'periodTime', 'eventType', 'shotType', 'coordinateX', 'coordinateY', 'emptynet', 'eventTeamId', 
            'teamHomeId',  'teamAwayId', 'away_skaters', 'home_skaters','zoneCode', 'lastEvent', 'lastCoordinateX', 'lastCoordinateY', 'lastDistance','timeLastEvent', 'speed', 'rebound'] 
        return pd.DataFrame(events_list, columns=columns)
    
    @staticmethod
    def clean_data(folder_path):
        all_events = []
        json_files = sorted(filter(lambda x: x.endswith('.json'), os.listdir(folder_path)))
    
        for filename in json_files:
            file_path = os.path.join(folder_path, filename)
            print(f"Traitement du fichier : {file_path}")
            try:
                extractor = NHLPlayByPlayExtractor(file_path)
                df = extractor.extract_events()
                all_events.append(df)
            except Exception as e:
                print(f"Erreur lors du traitement du fichier {file_path} : {e}")
    
        if all_events:
            final_df = pd.concat(all_events, ignore_index=True)
            return final_df
        else:
            return pd.DataFrame()

json_directory_path = 'nhl_data'
df2 = NHLPlayByPlayExtractor.clean_data(json_directory_path)
print(df2)




Traitement du fichier : nhl_data\2016020001.json
Traitement du fichier : nhl_data\2016020002.json
Traitement du fichier : nhl_data\2016020003.json
Traitement du fichier : nhl_data\2016020004.json
Traitement du fichier : nhl_data\2016020005.json
Traitement du fichier : nhl_data\2016020006.json
Traitement du fichier : nhl_data\2016020007.json
Traitement du fichier : nhl_data\2016020008.json
Traitement du fichier : nhl_data\2016020009.json
Traitement du fichier : nhl_data\2016020010.json
Traitement du fichier : nhl_data\2016020011.json
Traitement du fichier : nhl_data\2016020012.json
Traitement du fichier : nhl_data\2016020013.json
Traitement du fichier : nhl_data\2016020014.json
Traitement du fichier : nhl_data\2016020015.json
Traitement du fichier : nhl_data\2016020016.json
Traitement du fichier : nhl_data\2016020017.json
Traitement du fichier : nhl_data\2016020018.json
Traitement du fichier : nhl_data\2016020019.json
Traitement du fichier : nhl_data\2016020020.json
Traitement du fichie

In [153]:
df2.dtypes

gameId               int64
season               int64
period               int64
periodTime          object
eventType           object
shotType            object
coordinateX        float64
coordinateY        float64
emptynet             int64
eventTeamId          int64
teamHomeId           int64
teamAwayId           int64
away_skaters         int64
home_skaters         int64
zoneCode            object
lastEvent           object
lastCoordinateX      int64
lastCoordinateY      int64
lastDistance       float64
timeLastEvent        int64
speed              float64
rebound               bool
dtype: object

In [138]:
# See which columns have null values now
df2.columns[df2.isnull().any()].tolist()

['shotType', 'coordinateX', 'coordinateY', 'zoneCode']

In [154]:
df2.isnull().sum(axis = 0)

gameId              0
season              0
period              0
periodTime          0
eventType           0
shotType           36
coordinateX        16
coordinateY        15
emptynet            0
eventTeamId         0
teamHomeId          0
teamAwayId          0
away_skaters        0
home_skaters        0
zoneCode           14
lastEvent           0
lastCoordinateX     0
lastCoordinateY     0
lastDistance        0
timeLastEvent       0
speed               0
rebound             0
dtype: int64

In [140]:

print(df2.head)


<bound method NDFrame.head of             gameId    season  period periodTime     eventType  shotType  \
0       2016020001  20162017       1      01:11  shot-on-goal     wrist   
1       2016020001  20162017       1      02:53  shot-on-goal     wrist   
2       2016020001  20162017       1      04:01  shot-on-goal     wrist   
3       2016020001  20162017       1      04:46  shot-on-goal      slap   
4       2016020001  20162017       1      06:46  shot-on-goal     wrist   
...            ...       ...     ...        ...           ...       ...   
385071  2020030415  20202021       3      16:19  shot-on-goal     wrist   
385072  2020030415  20202021       3      17:13  shot-on-goal  backhand   
385073  2020030415  20202021       3      19:01  shot-on-goal     wrist   
385074  2020030415  20202021       3      19:10  shot-on-goal  backhand   
385075  2020030415  20202021       3      19:24  shot-on-goal     wrist   

        coordinateX  coordinateY  emptynet  eventTeamId  ...  away_sk

In [141]:
print(df2[['eventType','lastEvent']].head(20))

       eventType     lastEvent
0   shot-on-goal  blocked-shot
1   shot-on-goal      giveaway
2   shot-on-goal   missed-shot
3   shot-on-goal   missed-shot
4   shot-on-goal           hit
5   shot-on-goal      giveaway
6           goal   missed-shot
7   shot-on-goal       faceoff
8   shot-on-goal       faceoff
9   shot-on-goal       faceoff
10  shot-on-goal  shot-on-goal
11          goal  shot-on-goal
12  shot-on-goal  blocked-shot
13  shot-on-goal   missed-shot
14          goal  shot-on-goal
15          goal           hit
16  shot-on-goal       faceoff
17  shot-on-goal  blocked-shot
18  shot-on-goal       faceoff
19  shot-on-goal       faceoff


In [194]:
# Supprimer les lignes contenant des NaN dans les colonnes 'coordinateX' et 'coordinateY'
df2 = df2.dropna(axis=0, subset=['coordinateX', 'coordinateY', 'shotType'])

# Réinitialiser l'index après suppression des lignes
df2 = df2.reset_index(drop=True)

# Vérifier s'il reste des colonnes contenant des NaN
colonnes_avec_nan = df2.columns[df2.isnull().any()].tolist()
print("Colonnes avec des NaN après nettoyage :", colonnes_avec_nan)

Colonnes avec des NaN après nettoyage : []


In [157]:
df2.isnull().sum(axis = 0)

gameId             0
season             0
period             0
periodTime         0
eventType          0
shotType           0
coordinateX        0
coordinateY        0
emptynet           0
eventTeamId        0
teamHomeId         0
teamAwayId         0
away_skaters       0
home_skaters       0
zoneCode           0
lastEvent          0
lastCoordinateX    0
lastCoordinateY    0
lastDistance       0
timeLastEvent      0
speed              0
rebound            0
dtype: int64

In [142]:
print(df2.shape[0])

385076


In [158]:
print(df2.columns)

Index(['gameId', 'season', 'period', 'periodTime', 'eventType', 'shotType',
       'coordinateX', 'coordinateY', 'emptynet', 'eventTeamId', 'teamHomeId',
       'teamAwayId', 'away_skaters', 'home_skaters', 'zoneCode', 'lastEvent',
       'lastCoordinateX', 'lastCoordinateY', 'lastDistance', 'timeLastEvent',
       'speed', 'rebound'],
      dtype='object')


In [57]:

print(df2['season'].unique())

[20162017 20172018 20182019 20192020 20202021]


In [ ]:
import math
import pandas as pd
import numpy as np

def calcul_distance_angle(df):
   # Initialiser la liste des distances et le dictionnaire des côtés défensifs par période pour chaque ligne
    distances = []
    angles = []
    home_defending_side_period = {1: None, 2: None, 3: None}
    angle=None
    distance=None
    
    # Boucler sur chaque ligne pour calculer les distances
    
    for index, row in df.iterrows():
        # Initialiser la distance à None pour chaque ligne
        

        # Extraire les informations pertinentes
        x = row['coordinateX']
        y = row['coordinateY']
        event_team_id = row['eventTeamId']
        home_team_id = row['teamHomeId']
        defending_side = row.get('homeTeamDefendingSide', None)
        zone_code = row.get('zoneCode', None)
        period = row.get('period', None)
        season = row.get('season', None)

        # Vérifier la saison et définir les conditions de calcul
        if zone_code is not None:
            if event_team_id == home_team_id and period in home_defending_side_period:
                if home_defending_side_period[period] is None:
                    home_defending_side_period[period] = 'right' if x > 0 else 'left'
        
            if zone_code == "D":
                if x > 0:
                    distance = math.sqrt((x + 89)**2 + y**2)
                    angle = np.arctan2(y,(x+89))
                elif x < 0:
                    distance = math.sqrt((89 - x)**2 + y**2)
                    angle = np.arctan2(y,(89-x))
            elif zone_code == "O":
                if x > 0:
                    distance = math.sqrt((89 - x)**2 + y**2)
                    angle=np.arctan2(y,(89-x))
                elif x < 0:
                    distance = math.sqrt((x + 89)**2 + y**2)
                    angle= np.arctan2(y,(x+89))
            elif zone_code == "N" and period in home_defending_side_period: 
                if event_team_id == home_team_id:
                    if home_defending_side_period[period] == 'right':
                        distance = math.sqrt((x + 89)**2 + y**2)
                        angle= np.arctan2(y,(x+89))
                    elif home_defending_side_period[period] == 'left':
                        distance = math.sqrt((89 - x)**2 + y**2)
                        angle= np.arctan2(y,(89-x))
                else:
                    if home_defending_side_period[period] == 'right':
                        distance = math.sqrt((89 - x)**2 + y**2)
                        angle= np.arctan2(y,(89-x))
                    elif home_defending_side_period[period] == 'left':
                        distance = math.sqrt((x + 89)**2 + y**2)
                        angle= np.arctan2(y,(x+89))
        
        if angle is not None:
            angle = round(np.rad2deg(angle), 4)

        distances.append(distance)
        angles.append(angle)

    # Ajouter les distances calculées au DataFrame
    df['distance'] = distances
    # Ajouter les distances calculées au DataFrame
    df['angle'] = angles
    
    return df


# Calculer les distances pour toutes les saisons
df3 = calcul_distance_angle(df2)

# Afficher le DataFrame mis à jour
print(df3.tail())

            gameId    season  period periodTime     eventType  shotType  \
385020  2020030415  20202021       3      16:19  shot-on-goal     wrist   
385021  2020030415  20202021       3      17:13  shot-on-goal  backhand   
385022  2020030415  20202021       3      19:01  shot-on-goal     wrist   
385023  2020030415  20202021       3      19:10  shot-on-goal  backhand   
385024  2020030415  20202021       3      19:24  shot-on-goal     wrist   

        coordinateX  coordinateY  emptynet  eventTeamId  ...  zoneCode  \
385020         57.0         30.0         0           14  ...         O   
385021         56.0        -31.0         0           14  ...         O   
385022        -45.0        -14.0         0            8  ...         O   
385023        -77.0         -3.0         0            8  ...         O   
385024        -77.0         18.0         0            8  ...         O   

           lastEvent  lastCoordinateX  lastCoordinateY lastDistance  \
385020  shot-on-goal             

In [160]:
print(df3.columns)

Index(['gameId', 'season', 'period', 'periodTime', 'eventType', 'shotType',
       'coordinateX', 'coordinateY', 'emptynet', 'eventTeamId', 'teamHomeId',
       'teamAwayId', 'away_skaters', 'home_skaters', 'zoneCode', 'lastEvent',
       'lastCoordinateX', 'lastCoordinateY', 'lastDistance', 'timeLastEvent',
       'speed', 'rebound', 'distance', 'angle'],
      dtype='object')


In [196]:
df3.isnull().sum(axis = 0)

gameId             0
season             0
period             0
periodTime         0
eventType          0
shotType           0
coordinateX        0
coordinateY        0
emptynet           0
eventTeamId        0
teamHomeId         0
teamAwayId         0
away_skaters       0
home_skaters       0
zoneCode           0
lastEvent          0
lastCoordinateX    0
lastCoordinateY    0
lastDistance       0
timeLastEvent      0
speed              0
rebound            0
distance           0
angle              0
dtype: int64

In [197]:
# Calculer les secondes de jeu en fonction de la période et du temps dans la période
def calculate_game_seconds(row):
    minutes, seconds = map(int, row['periodTime'].split(':'))
    return (row['period'] - 1) * 1200 + minutes * 60 + seconds


print(df3.shape[0])

df3.loc[:, 'gameSeconds'] = df3.apply(calculate_game_seconds, axis=1)

print(df3.shape[0])
# Afficher le DataFrame mis à jour
print(df3.tail())

385025
385025
            gameId    season  period periodTime     eventType  shotType  \
385020  2020030415  20202021       3      16:19  shot-on-goal     wrist   
385021  2020030415  20202021       3      17:13  shot-on-goal  backhand   
385022  2020030415  20202021       3      19:01  shot-on-goal     wrist   
385023  2020030415  20202021       3      19:10  shot-on-goal  backhand   
385024  2020030415  20202021       3      19:24  shot-on-goal     wrist   

        coordinateX  coordinateY  emptynet  eventTeamId  ...     lastEvent  \
385020         57.0         30.0         0           14  ...  shot-on-goal   
385021         56.0        -31.0         0           14  ...           hit   
385022        -45.0        -14.0         0            8  ...       faceoff   
385023        -77.0         -3.0         0            8  ...  shot-on-goal   
385024        -77.0         18.0         0            8  ...  shot-on-goal   

        lastCoordinateX  lastCoordinateY  lastDistance timeLastEve

In [198]:
print (df3.columns)
# Filter rows where rebound is True
rebound_rows = df3[df3['rebound'] == True]

# Display the filtered DataFrame
print(rebound_rows.tail())

# Optionally, show the number of rows with rebounds
print("Number of rebounds:", rebound_rows.shape[0])

Index(['gameId', 'season', 'period', 'periodTime', 'eventType', 'shotType',
       'coordinateX', 'coordinateY', 'emptynet', 'eventTeamId', 'teamHomeId',
       'teamAwayId', 'away_skaters', 'home_skaters', 'zoneCode', 'lastEvent',
       'lastCoordinateX', 'lastCoordinateY', 'lastDistance', 'timeLastEvent',
       'speed', 'rebound', 'distance', 'angle', 'gameSeconds'],
      dtype='object')
            gameId    season  period periodTime     eventType  shotType  \
385013  2020030415  20202021       3      09:03  shot-on-goal      snap   
385018  2020030415  20202021       3      12:28  shot-on-goal     wrist   
385020  2020030415  20202021       3      16:19  shot-on-goal     wrist   
385023  2020030415  20202021       3      19:10  shot-on-goal  backhand   
385024  2020030415  20202021       3      19:24  shot-on-goal     wrist   

        coordinateX  coordinateY  emptynet  eventTeamId  ...     lastEvent  \
385013         82.0          2.0         0           14  ...  shot-on-goal 

Changement d'angle de tir et vitesse de changement

In [199]:
import pandas as pd
import numpy as np

# Supposez que votre DataFrame s'appelle `df` et qu'il contient les colonnes nécessaires : 'angle', 'time', 'rebound'
df3['chang_angle'] = 0  # Initialisez avec 0
df3['vitesse_chang_angle'] = 0  # Initialisez avec 0

# Boucle pour calculer le changement d'angle et la vitesse
for i in range(1, len(df3)):
    if df3.loc[i, 'rebound']:  # Si c'est un rebond
        angle_actuel = df3.loc[i, 'angle']
        angle_precedent = df3.loc[i-1, 'angle']
        temps_ecoule = df3.loc[i, 'timeLastEvent']
        
        # Calcul du changement d'angle
        changement_angle = abs(180-(angle_actuel +angle_precedent))
        df3.loc[i, 'chang_angle'] = changement_angle
        
        # Calcul de la vitesse du changement d'angle
        if temps_ecoule:  
              df3.loc[i, 'vitesse_chang_angle'] = changement_angle / temps_ecoule
        else:
              df3.loc[i, 'vitesse_chang_angle'] =0

print(df3.tail())

C:\Users\user\AppData\Local\Temp\ipykernel_17948\225240626.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '184.4609' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df3.loc[i, 'chang_angle'] = changement_angle
C:\Users\user\AppData\Local\Temp\ipykernel_17948\225240626.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0021351865356344993' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df3.loc[i, 'vitesse_chang_angle'] = changement_angle / temps_ecoule


            gameId    season  period periodTime     eventType  shotType  \
385020  2020030415  20202021       3      16:19  shot-on-goal     wrist   
385021  2020030415  20202021       3      17:13  shot-on-goal  backhand   
385022  2020030415  20202021       3      19:01  shot-on-goal     wrist   
385023  2020030415  20202021       3      19:10  shot-on-goal  backhand   
385024  2020030415  20202021       3      19:24  shot-on-goal     wrist   

        coordinateX  coordinateY  emptynet  eventTeamId  ...  lastCoordinateY  \
385020         57.0         30.0         0           14  ...               35   
385021         56.0        -31.0         0           14  ...               40   
385022        -45.0        -14.0         0            8  ...              -22   
385023        -77.0         -3.0         0            8  ...              -14   
385024        -77.0         18.0         0            8  ...               -3   

        lastDistance  timeLastEvent     speed rebound   distan

Sauvegarde dans un fichier pour exploitation ulterieure

In [200]:
print(df3.head)

<bound method NDFrame.head of             gameId    season  period periodTime     eventType  shotType  \
0       2016020001  20162017       1      01:11  shot-on-goal     wrist   
1       2016020001  20162017       1      02:53  shot-on-goal     wrist   
2       2016020001  20162017       1      04:01  shot-on-goal     wrist   
3       2016020001  20162017       1      04:46  shot-on-goal      slap   
4       2016020001  20162017       1      06:46  shot-on-goal     wrist   
...            ...       ...     ...        ...           ...       ...   
385020  2020030415  20202021       3      16:19  shot-on-goal     wrist   
385021  2020030415  20202021       3      17:13  shot-on-goal  backhand   
385022  2020030415  20202021       3      19:01  shot-on-goal     wrist   
385023  2020030415  20202021       3      19:10  shot-on-goal  backhand   
385024  2020030415  20202021       3      19:24  shot-on-goal     wrist   

        coordinateX  coordinateY  emptynet  eventTeamId  ...  lastCoo

Ingenieurie des caracteristiques

In [201]:
print(df3.isnull().sum(axis = 0))
print(df3.shape[0])

gameId                 0
season                 0
period                 0
periodTime             0
eventType              0
shotType               0
coordinateX            0
coordinateY            0
emptynet               0
eventTeamId            0
teamHomeId             0
teamAwayId             0
away_skaters           0
home_skaters           0
zoneCode               0
lastEvent              0
lastCoordinateX        0
lastCoordinateY        0
lastDistance           0
timeLastEvent          0
speed                  0
rebound                0
distance               0
angle                  0
gameSeconds            0
chang_angle            0
vitesse_chang_angle    0
dtype: int64
385025


Creer la colonne cible isGoal

In [202]:
isGoal = df3['eventType'] == 'goal'
df3['isGoal'] = isGoal.astype(int)  # Add this line to create the 'isGoal' column


In [203]:
print(df3.columns)
print(df3[['eventType', 'isGoal']])

Index(['gameId', 'season', 'period', 'periodTime', 'eventType', 'shotType',
       'coordinateX', 'coordinateY', 'emptynet', 'eventTeamId', 'teamHomeId',
       'teamAwayId', 'away_skaters', 'home_skaters', 'zoneCode', 'lastEvent',
       'lastCoordinateX', 'lastCoordinateY', 'lastDistance', 'timeLastEvent',
       'speed', 'rebound', 'distance', 'angle', 'gameSeconds', 'chang_angle',
       'vitesse_chang_angle', 'isGoal'],
      dtype='object')
           eventType  isGoal
0       shot-on-goal       0
1       shot-on-goal       0
2       shot-on-goal       0
3       shot-on-goal       0
4       shot-on-goal       0
...              ...     ...
385020  shot-on-goal       0
385021  shot-on-goal       0
385022  shot-on-goal       0
385023  shot-on-goal       0
385024  shot-on-goal       0

[385025 rows x 2 columns]


In [204]:
print(df3['shotType'].unique())
print(df3['shotType'].value_counts())
print(df3['lastEvent'].unique())
print(df3['lastEvent'].value_counts())

['wrist' 'slap' 'backhand' 'snap' 'tip-in' 'deflected' 'wrap-around']
shotType
wrist          210090
slap            58761
snap            54838
backhand        31049
tip-in          19681
deflected        6753
wrap-around      3853
Name: count, dtype: int64
['blocked-shot' 'giveaway' 'missed-shot' 'hit' 'faceoff' 'shot-on-goal'
 'stoppage' 'takeaway' 'penalty' 'goal' 'period-start' 'period-end'
 'game-end' 'delayed-penalty']
lastEvent
faceoff            93930
shot-on-goal       71909
hit                68924
blocked-shot       40024
giveaway           35391
missed-shot        35339
takeaway           25442
stoppage           11113
goal                1002
penalty              766
delayed-penalty      521
period-start         406
period-end           253
game-end               5
Name: count, dtype: int64


Changement des caracteristiques categorielles en numeriques

In [205]:
from sklearn.preprocessing import LabelEncoder
# creer une instance de labelencoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column

# Make an explicit copy of df_final2
df4 = df3.copy()

# Apply label encoding
df4['shotType'] = labelencoder.fit_transform(df4['shotType'])
df4['lastEvent'] = labelencoder.fit_transform(df4['lastEvent'])
df4

,gameId,season,period,periodTime,eventType,shotType,coordinateX,coordinateY,emptynet,eventTeamId,...,lastDistance,timeLastEvent,speed,rebound,distance,angle,gameSeconds,chang_angle,vitesse_chang_angle,isGoal
0,2016020001,20162017,1,01:11,shot-on-goal,6,-77.0,5.0,0,10,...,17.088007,86399,0.000198,False,13.000000,22.6199,71,0.0000,0.000000,0
1,2016020001,20162017,1,02:53,shot-on-goal,6,86.0,13.0,0,9,...,36.715120,86395,0.000425,False,13.341664,77.0054,173,0.0000,0.000000,0
2,2016020001,20162017,1,04:01,shot-on-goal,6,23.0,-38.0,0,9,...,102.318131,86382,0.001184,False,118.270876,-18.7413,241,0.0000,0.000000,0
3,2016020001,20162017,1,04:46,shot-on-goal,2,33.0,-15.0,0,9,...,45.880279,86381,0.000531,False,57.974132,-14.9951,286,0.0000,0.000000,0
4,2016020001,20162017,1,06:46,shot-on-goal,6,-34.0,28.0,0,10,...,81.221918,86384,0.000940,False,61.717096,26.9802,406,0.0000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385020,2020030415,20202021,3,16:19,shot-on-goal,6,57.0,30.0,0,14,...,128.097619,86385,0.001483,True,43.863424,43.1524,3379,74.0637,0.000857,0
385021,2020030415,20202021,3,17:13,shot-on-goal,0,56.0,-31.0,0,14,...,71.847060,86391,0.000832,False,45.276926,-43.2101,3433,0.0000,0.000000,0
385022,2020030415,20202021,3,19:01,shot-on-goal,6,-45.0,-14.0,0,8,...,25.298221,86375,0.000293,False,46.173586,-17.6501,3541,0.0000,0.000000,0
385023,2020030415,20202021,3,19:10,shot-on-goal,0,-77.0,-3.0,0,8,...,33.837849,86391,0.000392,True,12.369317,-14.0362,3550,211.6863,0.002450,0


In [206]:
print(df4['shotType'].unique())
print(df4['shotType'].value_counts())
print(df4['lastEvent'].unique())
print(df4['lastEvent'].value_counts())

[6 2 0 3 4 1 5]
shotType
6    210090
2     58761
3     54838
0     31049
4     19681
1      6753
5      3853
Name: count, dtype: int64
[ 0  4  7  6  2 11 12 13  8  5 10  9  3  1]
lastEvent
2     93930
11    71909
6     68924
0     40024
4     35391
7     35339
13    25442
12    11113
5      1002
8       766
1       521
10      406
9       253
3         5
Name: count, dtype: int64


Transformer la caracteristique rebound en valeur numérique

In [207]:

df4["rebound"] = df2["rebound"].astype(int)  # Remplacer les valeurs false and true par 0 et 1
print(df4["rebound"] )
print(df4.tail())



0         0
1         0
2         0
3         0
4         0
         ..
385020    1
385021    0
385022    0
385023    1
385024    1
Name: rebound, Length: 385025, dtype: int32
            gameId    season  period periodTime     eventType  shotType  \
385020  2020030415  20202021       3      16:19  shot-on-goal         6   
385021  2020030415  20202021       3      17:13  shot-on-goal         0   
385022  2020030415  20202021       3      19:01  shot-on-goal         6   
385023  2020030415  20202021       3      19:10  shot-on-goal         0   
385024  2020030415  20202021       3      19:24  shot-on-goal         6   

        coordinateX  coordinateY  emptynet  eventTeamId  ...  lastDistance  \
385020         57.0         30.0         0           14  ...    128.097619   
385021         56.0        -31.0         0           14  ...     71.847060   
385022        -45.0        -14.0         0            8  ...     25.298221   
385023        -77.0         -3.0         0            8  ... 

In [83]:
df4.describe()

,gameId,season,period,shotType,coordinateX,coordinateY,emptynet,eventTeamId,teamHomeId,lastEvent,lastDistance,timeLastEvent,speed,rebound,distance,angle,gameSeconds,chang_angle,vitesse_chang_angle,isGoal
count,3.850250e+05,3.850250e+05,385025.000000,385025.000000,385025.000000,385025.000000,385025.000000,385025.000000,385025.000000,385025.000000,385025.000000,385025.000000,385025.000000,385025.000000,385025.000000,385025.000000,385025.000000,385025.000000,385025.000000,385025.000000
mean,2.017869e+09,2.018050e+07,2.047353,4.278470,-0.245213,0.057108,0.004631,18.678883,18.672385,5.878717,60.704694,81273.667076,0.195076,0.186764,36.333516,-0.698129,1847.257373,33.899933,0.422556,0.094903
std,1.352748e+06,1.352861e+04,0.877201,2.077521,63.510005,18.955603,0.067893,13.688373,13.708362,4.135665,47.050768,20377.634635,2.802462,0.389723,25.560286,345.963094,1081.851979,76.425586,8.269424,0.293081
min,2.016020e+09,2.016202e+07,1.000000,0.000000,-99.000000,-42.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-141458.753300,1.000000,0.000000,0.000000,0.000000
25%,2.017020e+09,2.017202e+07,1.000000,2.000000,-64.000000,-14.000000,0.000000,8.000000,8.000000,2.000000,24.698178,86375.000000,0.000280,0.000000,17.029386,-28.300800,931.000000,0.000000,0.000000,0.000000
50%,2.018020e+09,2.018202e+07,2.000000,6.000000,-4.000000,0.000000,0.000000,17.000000,17.000000,6.000000,45.705580,86388.000000,0.000531,0.000000,33.015148,0.000000,1820.000000,0.000000,0.000000,0.000000
75%,2.019021e+09,2.019202e+07,3.000000,6.000000,63.000000,14.000000,0.000000,25.000000,24.000000,11.000000,91.219515,86394.000000,0.001084,0.000000,49.517674,27.847600,2737.000000,0.000000,0.000000,0.000000
max,2.020030e+09,2.020202e+07,8.000000,6.000000,99.000000,42.000000,1.000000,54.000000,54.000000,13.000000,201.109423,86399.000000,187.523332,1.000000,189.654950,104609.906600,9027.000000,4656.633800,368.130100,1.000000


In [172]:
df4.dtypes

gameId                   int64
season                   int64
period                   int64
periodTime              object
eventType               object
shotType                 int32
coordinateX            float64
coordinateY            float64
emptynet                 int64
eventTeamId              int64
teamHomeId               int64
teamAwayId               int64
away_skaters             int64
home_skaters             int64
zoneCode                object
lastEvent                int32
lastCoordinateX          int64
lastCoordinateY          int64
lastDistance           float64
timeLastEvent            int64
speed                  float64
rebound                   bool
distance               float64
angle                  float64
gameSeconds              int64
chang_angle            float64
vitesse_chang_angle    float64
isGoal                   int32
dtype: object

Partie Bonus:

In [173]:
print(df4.columns)

Index(['gameId', 'season', 'period', 'periodTime', 'eventType', 'shotType',
       'coordinateX', 'coordinateY', 'emptynet', 'eventTeamId', 'teamHomeId',
       'teamAwayId', 'away_skaters', 'home_skaters', 'zoneCode', 'lastEvent',
       'lastCoordinateX', 'lastCoordinateY', 'lastDistance', 'timeLastEvent',
       'speed', 'rebound', 'distance', 'angle', 'gameSeconds', 'chang_angle',
       'vitesse_chang_angle', 'isGoal'],
      dtype='object')


In [208]:
def non_gardiens_amicaux (eventTeam,teamHome, home_skaters,away_skaters):
    """
    Takes data about the event and returns the number of friendly players on the ice.
    """
    # If the team taking the shot is the same as the home team (ie home = friendly):
    if eventTeam == teamHome:
        # Return the home players on ice == friendly players on ice
        return home_skaters
    else:
        # Otherwise, the team taking the shot is the away team, so away=friendly
        return away_skaters
    
def  non_gardiens_adverses (eventTeam,teamHome, home_skaters,away_skaters):
    """
    Takes data about the event and returns the number of opposing players on the ice.
    """
    # if the team taking the shot is the same team as the home team (ie. home = friendly, away = opposing):
    if eventTeam == teamHome:
        # return the away players on ice == opposing players on ice
        return away_skaters
    else:
        # Otherwise, the team taking the shot is the away team, so home = opposing and away = friendly
        return home_skaters

In [209]:
df4['non_gardiens_amicaux'] = [non_gardiens_amicaux(df4['eventTeamId'][i],
                                               df4['teamHomeId'][i],
                                              df4['home_skaters'][i],
                                              df4['away_skaters'][i]) for i,r in df4.iterrows()]
df4['non_gardiens_adverses'] = [non_gardiens_adverses(df4['eventTeamId'][i],
                                               df4['teamHomeId'][i],
                                              df4['home_skaters'][i],
                                              df4['away_skaters'][i]) for i,r in df4.iterrows()]

In [210]:
print(df4.columns)

Index(['gameId', 'season', 'period', 'periodTime', 'eventType', 'shotType',
       'coordinateX', 'coordinateY', 'emptynet', 'eventTeamId', 'teamHomeId',
       'teamAwayId', 'away_skaters', 'home_skaters', 'zoneCode', 'lastEvent',
       'lastCoordinateX', 'lastCoordinateY', 'lastDistance', 'timeLastEvent',
       'speed', 'rebound', 'distance', 'angle', 'gameSeconds', 'chang_angle',
       'vitesse_chang_angle', 'isGoal', 'non_gardiens_amicaux',
       'non_gardiens_adverses'],
      dtype='object')


In [212]:
print(df4.tail())


            gameId    season  period periodTime     eventType  shotType  \
385020  2020030415  20202021       3      16:19  shot-on-goal         6   
385021  2020030415  20202021       3      17:13  shot-on-goal         0   
385022  2020030415  20202021       3      19:01  shot-on-goal         6   
385023  2020030415  20202021       3      19:10  shot-on-goal         0   
385024  2020030415  20202021       3      19:24  shot-on-goal         6   

        coordinateX  coordinateY  emptynet  eventTeamId  ...     speed  \
385020         57.0         30.0         0           14  ...  0.001483   
385021         56.0        -31.0         0           14  ...  0.000832   
385022        -45.0        -14.0         0            8  ...  0.000293   
385023        -77.0         -3.0         0            8  ...  0.000392   
385024        -77.0         18.0         0            8  ...  0.000243   

        rebound   distance    angle gameSeconds  chang_angle  \
385020        1  43.863424  43.1524     

In [182]:
print(df4[['non_gardiens_amicaux','non_gardiens_adverses']])

        non_gardiens_amicaux  non_gardiens_adverses
0                          5                      5
1                          5                      5
2                          5                      5
3                          5                      5
4                          5                      5
...                      ...                    ...
385020                     5                      5
385021                     5                      5
385022                     6                      5
385023                     6                      5
385024                     6                      5

[385025 rows x 2 columns]


Reparatititon en donnees train and test

In [213]:
# Ensure 'gameId' is treated as a string
df4['gameId'] = df4['gameId'].astype(str)

# Extract year from 'gameId' (first four characters)
df4['gameYear'] = df4['gameId'].str[:4]

# Split the data into training and testing datasets
train_data = df4[df4['gameYear'].isin(['2016', '2017', '2018', '2019'])]
test_data = df4[df4['gameYear'] == '2020']

In [214]:
print(train_data.columns)
print(train_data['gameYear'].unique())
print(test_data['gameYear'].unique())

Index(['gameId', 'season', 'period', 'periodTime', 'eventType', 'shotType',
       'coordinateX', 'coordinateY', 'emptynet', 'eventTeamId', 'teamHomeId',
       'teamAwayId', 'away_skaters', 'home_skaters', 'zoneCode', 'lastEvent',
       'lastCoordinateX', 'lastCoordinateY', 'lastDistance', 'timeLastEvent',
       'speed', 'rebound', 'distance', 'angle', 'gameSeconds', 'chang_angle',
       'vitesse_chang_angle', 'isGoal', 'non_gardiens_amicaux',
       'non_gardiens_adverses', 'gameYear'],
      dtype='object')
['2016' '2017' '2018' '2019']
['2020']


In [216]:
#Check that train data contains only the right seasons
print(f'train data:')
print(train_data['gameId'].str[:4].unique())
#Check that test data contains only the right seasons
print(f'test data: ')
print(test_data['gameId'].str[:4].unique())

train data:
['2016' '2017' '2018' '2019']
test data: 
['2020']


In [217]:
# output to csv
df4.to_csv('Q4_data.csv', index=False)  
train_data.to_csv('Q4_train.csv', index=False)  
test_data.to_csv('Q4_test.csv',index=False)  

In [218]:
train_download = pd.read_csv('Q4_train.csv', dtype={10: str}, low_memory=False)
print(train_download.columns)

print(train_download['emptynet'].value_counts())
print(train_download['lastEvent'].value_counts())
print(train_download['rebound'].value_counts())
print(train_download['isGoal'].value_counts())

#print(df3.unique())

Index(['gameId', 'period', 'shotType', 'coordinateX', 'coordinateY',
       'emptynet', 'lastEvent', 'lastCoordinateX', 'lastCoordinateY',
       'lastDistance', 'timeLastEvent', 'speed', 'rebound', 'distance',
       'angle', 'gameSeconds', 'chang_angle', 'non_gardiens_amicaux',
       'non_gardiens_adverses', 'isGoal'],
      dtype='object')
emptynet
0    325800
1      1495
Name: count, dtype: int64
lastEvent
2     79729
11    60508
6     57638
0     33850
4     30070
7     30001
13    21858
12    11078
5       899
8       743
10      347
1       324
9       247
3         3
Name: count, dtype: int64
rebound
0    266787
1     60508
Name: count, dtype: int64
isGoal
0    296387
1     30908
Name: count, dtype: int64


Partie WANDB

In [295]:
df = pd.read_csv('Q4_data.csv', index_col=0)
print(df.columns)

Index(['period', 'shotType', 'coordinateX', 'coordinateY', 'emptynet',
       'lastEvent', 'lastCoordinateX', 'lastCoordinateY', 'lastDistance',
       'timeLastEvent', 'speed', 'rebound', 'distance', 'angle', 'gameSeconds',
       'chang_angle', 'vitesse_chang_angle', 'isGoal', 'gameYear'],
      dtype='object')


In [ ]:

# Filtrer les données pour le match demandé
filtered_match = df[df['gameId'] == 2017021065]

# Afficher un aperçu des données filtrées
filtered_match.info(), filtered_match.head()

<class 'pandas.core.frame.DataFrame'>
Index: 71 entries, 149066 to 149136
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   gameId                 71 non-null     int64  
 1   season                 71 non-null     int64  
 2   teamHomeId             71 non-null     int64  
 3   teamHome               71 non-null     object 
 4   teamAwayId             71 non-null     int64  
 5   teamAway               71 non-null     object 
 6   event_id               71 non-null     int64  
 7   eventType              71 non-null     object 
 8   eventTeamId            71 non-null     int64  
 9   eventTeam              71 non-null     object 
 10  homeTeamDefendingSide  0 non-null      object 
 11  zoneCode               71 non-null     object 
 12  period                 71 non-null     int64  
 13  periodTime             71 non-null     object 
 14  coordinateX            71 non-null     float64
 15  coor

(None,
             gameId    season  teamHomeId  teamHome  teamAwayId teamAway  \
 149066  2017021065  20172018          15  Capitals          52     Jets   
 149067  2017021065  20172018          15  Capitals          52     Jets   
 149068  2017021065  20172018          15  Capitals          52     Jets   
 149069  2017021065  20172018          15  Capitals          52     Jets   
 149070  2017021065  20172018          15  Capitals          52     Jets   
 
         event_id     eventType  eventTeamId eventTeam  ... zoneCode period  \
 149066        13  shot-on-goal           15  Capitals  ...        O      1   
 149067        14  shot-on-goal           15  Capitals  ...        O      1   
 149068        16  shot-on-goal           52      Jets  ...        O      1   
 149069        17  shot-on-goal           15  Capitals  ...        O      1   
 149070        18  shot-on-goal           52      Jets  ...        N      1   
 
         periodTime coordinateX  coordinateY    shooterName

Sur CMD: set WANDB_API_KEY=79080fa86c35e3d0b207c46caabf5b8b8f5a9288

Puis dans le code: wandb.login(key=os.getenv("WANDB_API_KEY"))



In [ ]:
import wandb
import pandas as pd
import os

# Récupérer la clé API à partir des variables d'environnement
wandb.login(key=os.getenv("WANDB_API_KEY"))

# Initialiser un projet wandb
wandb.init(
    project="ingenieurie_char_data",  # Nom du projet
    entity="thalia-cantero-udem",    # Nom de votre espace de travail
    config={"dataset": "Q4_train.csv"}  # Configuration facultative
)



# Filtrer pour obtenir uniquement le jeu de données requis
subset_df =  df[df['gameId'] == 2017021065]

# Enregistrer le DataFrame dans wandb
wandb.log({"wpg_v_wsh_2017021065": wandb.Table(dataframe=subset_df)})

# Fin de l'exécution
wandb.finish()


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Wandb fournit certaines fonctionnalités pour suivre les ensembles de données, soit en tant qu'artefacts bruts que vous enregistrez manuellement, soit en téléchargeant automatiquement des DataFrame/tables pour vous. Il dispose également de fonctionnalités intéressantes pour calculer les hashs de jeux de données, vous permettant de vérifier si vous vous entraînez sur les mêmes données que les autres. Vous pouvez trouver plus d'informations sur cette fonctionnalité dans leur documentation. Vous n'êtes pas obligé d'utiliser cette fonctionnalité pour la majorité de votre projet (vous pouvez si vous le souhaitez !). Cependant, aux fins de cette section, nous vous demandos de soumettre un petit sous-ensemble (un jeu spécifique) de votre cadre de données final avec toutes vos caractéristiques finales. Filtrez votre ensemble de données sur le match Winnipeg vs Washington le 12 mars 2018. Ce jeu a l'ID de jeu "2017021065".  Téléchargez le DataFrame filtré avec toutes les caractéristiques que vous avez créées en tant que CSV à l'aide de la classe  Table ; gardez le nom comme 'wpg_v_wsh_2017021065'.  



In [ ]:
import wandb
import pandas as pd
import os
# Connexion à WandB
wandb.login(key=os.getenv("WANDB_API_KEY"))

# Initialiser un projet WandB
run = wandb.init(
    project="ingenieurie_char_data",  # Nom du projet
    entity="thalia-cantero-udem",    # Nom de votre espace de travail
    config={"dataset": "Q4_train.csv"}  # Configuration facultative
)

# Filtrer le DataFrame pour le match Winnipeg vs Washington
game_id = 2017021065
subset_df = df3[df3['gameId'] == game_id]

# Vérifiez que le DataFrame n'est pas vide
if subset_df.empty:
    raise ValueError(f"Le DataFrame filtré pour le gameId {game_id} est vide. Veuillez vérifier vos données.")

# Créer un artefact pour enregistrer le DataFrame
artifact = wandb.Artifact(
    name="wpg_v_wsh_2017021065",  # Nom de l'artefact
    type="dataset",               # Type de l'artefact
    description="Sous-ensemble du DataFrame filtré pour le match Winnipeg vs Washington le 12 mars 2018."
)
# Ajouter le DataFrame au format table
artifact.add(wandb.Table(dataframe=subset_df), "filtered_table")

# Journaliser l'artefact
run.log_artifact(artifact)

# Fin de l'exécution
wandb.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: khalida-g (thalia-cantero-udem). Use `wandb login --relogin` to force relogin
